# Kernel Machines

## Test multiclass SVM - without kernel

In [ ]:
from metric import * 

from svm import *

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


# Toy data 생성
np.random.seed(42)
X = np.random.randn(1500, 2)
y =  np.repeat([0, 1, 2], 500)

X[y == 0] += -5.0
X[y == 1] += 5.0
X[y == 2] += 2.5

# Data 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

def one_hot_list(n, k): # Helper - index to one-hot vector
    tensor = n * [0]
    tensor[k] = 1
    return tensor

y_train = [one_hot_list(3, y) for y in y_train]
y_train = torch.tensor(y_train)

y_test = [one_hot_list(3, y) for y in y_test]
y_test = torch.tensor(y_test)

###### 커널 ###
def someRandomKernel(x):
    return torch.cat((
            x, torch.tensor(
                 [x[0] + x[1], x[1] * x[1]] 
                )
    ))
identity = lambda x : x
####

kernelToMatrix = lambda x : torch.stack([someRandomKernel(x[i]) for i in range(x.size()[0])])
print("X_train.size()", X_train.size())
print("X_train.kernel()", kernelToMatrix(X_train).size())


In [ ]:

#kernel을 바꿀 경우 input_dim을 바꿔야 한다
model = KernelSVM(input_dim=2, num_classes=3, kernel=identity , C=1.0) 
model.train(X_train, y_train, epochs=1000)

train_predictions = model.predict(X_train, using=KernelSVM.predict_index)
test_predictions = model.predict(X_test, using=KernelSVM.predict_index)

########################################################################
###### make as index-based classified tensor to compare naturally ######
y_test = torch.tensor([torch.argmax(y_test[i]) for i in range(y_test.size()[0])])
y_train = torch.tensor([torch.argmax(y_train[i]) for i in range(y_train.size()[0])])
########################################################################

train_accuracy = torch.sum(train_predictions == y_train) / y_train.numel()
test_accuracy = torch.sum(test_predictions == y_test) / y_test.numel()


print(f"Training Accuracy: {train_accuracy*100:.2f}%")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print("Precision - micro is ",precision_micro(num_classes=model.num_classes, predict=test_predictions, truth=y_test, using=REPRESENT.INDEX))
print("Precision - macro is ",precision_macro(num_classes=model.num_classes, predict=test_predictions, truth=y_test, using=REPRESENT.INDEX))
print("Recall - micro is ",recall_micro(num_classes=model.num_classes, predict=test_predictions, truth=y_test, using=REPRESENT.INDEX))
print("Recall - micro is ",recall_macro(num_classes=model.num_classes, predict=test_predictions, truth=y_test, using=REPRESENT.INDEX))

print("F1-score is ",f1score(num_classes=model.num_classes, predict=test_predictions, truth=y_test, precision=precision_macro, recall=recall_macro, using=REPRESENT.INDEX))

X_test_np = X_test.detach().numpy() if torch.is_tensor(X_test) else X_test
y_test_np = y_test.detach().numpy() if torch.is_tensor(y_test) else y_test
test_predictions_np = test_predictions.detach().numpy() if torch.is_tensor(test_predictions) else test_predictions

#y_test_np_indexed = np.argmax(y_test_np, axis=1)
#test_predictions_np_indexed = np.argmax(test_predictions_np, axis=1)

# 같은 클래스로 분류를 성공했음에도 색깔은 바꿔서 표시되는 이슈가 있음
plt.scatter(X_test_np[:, 0], X_test_np[:, 1], c=y_test_np, cmap='jet', marker='o', label='True classes', alpha=0.5)
plt.scatter(X_test_np[:, 0], X_test_np[:, 1], c=test_predictions_np, cmap='jet', marker='x', label='Predicted classes', alpha=0.5)
plt.legend()
plt.show()




In [ ]:
model.predict(torch.tensor([[3.0,3]]))

In [ ]:
model.predict(torch.tensor([[3.0,3], [0.0,0], [-1.0,1]]), using=KernelSVM.predict_index)